In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set random seeds for reproducibility
torch.manual_seed(42)

# Define transformation for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# Load CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Define the classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
import torch.nn as nn

class RBFNet(nn.Module):
    def __init__(self, num_features, num_hidden, num_classes):
        super(RBFNet, self).__init__()
        
        # Define the RBF hidden layer
        self.hidden = nn.Linear(num_features, num_hidden)
        self.hidden_activation = nn.RBF(num_hidden, 2)
        
        # Define the output layer
        self.output = nn.Linear(num_hidden, num_classes)
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.hidden_activation(x)
        x = self.output(x)
        return x

# Create an instance of the RBFNet model
net = RBFNet(num_features=3072, num_hidden=20, num_classes=10)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data

        # Flatten the inputs
        inputs = inputs.view(inputs.size(0), -1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished training')


In [ ]:
correct_train = 0
total_train = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images = images.view(images.size(0), -1)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

train_accuracy = 100 * correct_train / total_train
print('Training Accuracy: %.2f %%' % train_accuracy)

correct_test = 0
total_test = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.view(images.size(0), -1)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print('Test Accuracy: %.2f %%' % test_accuracy)
